# Problem:
We want to detect the area between the nose and the mouth in the keypoints and place the mustache there.

Imports

In [1]:
import cv2
import numpy as np
import time
from math import sin, cos, radians

Getting the classifier for the face. Also getting the mustache we want to put on our face

In [2]:
cascPath = "detector_architectures/haarcascade_frontalface_default.xml"  # for face detection
faceCascade = cv2.CascadeClassifier(cascPath)

mst = cv2.imread('images/mustache.png')

Here, we define the put mustache function.

It accepts:
    * the moustache
    * the face
    * The x and y 
    * The width and height of the face
    
There are a bunch of hardcoded constants in here. It's pretty unfortunate, but there's nothing we can do about them. (Took me a lot of time to figure these out myself)

In [3]:
def put_mustache(mst,fc,x,y,w,h):
    
    #Getting the width and height of the face
    face_width = w
    face_height = h
    
    #Get the width and height which we want to scale the mustache down to(makes this scalable)
    mst_width = int(face_width*0.42)+1 #Certain scalable constant, looks decent
    mst_height = int(face_height*0.14)+1 #Certain scalable constant, looks decent

    #Resizing the mustache
    mst = cv2.resize(mst,(mst_width,mst_height)) 
    
    #Here comes the pain and the constants. MUST BE HARDCODED FOR NO ERRORS!
    #For each y pixel in the face in the general area where we want to put the mustache
    for i in range(int(0.62857142857*face_height),int(0.62857142857*face_height)+mst_height):
        # For each row of pixels in that y value
        for j in range(int(0.29166666666*face_width),int(0.29166666666*face_width)+mst_width):
            # For 3 keypoints
            for k in range(3):
                # if they are not out of the box we want to put them
                if mst[i-int(0.62857142857*face_height)][j-int(0.29166666666*face_width)][k] <235:
                    # Place certain sections of the mustache on the face at the proper keypoints
                    fc[y+i][x+j][k] = mst[i-int(0.62857142857*face_height)][j-int(0.29166666666*face_width)][k]
    return fc

Helper methods:
* Rotating the image a certain *angle* degrees:
    - Getting the width and height
    - Creating a rotation matrix using cv2's function
    - Warping the new image using the rotation matrix
    - returning that warped image
* Rotating the point in the image by a certain *angle* degrees
    - Used for detecting the face even when it is slightly tilted
    - Get the x and y for 0.6 times image (used in calculating new x)
    - Calculating the new x and y by using fancy math (yey)
    - Returning the pixel values for the new x and y, then the other default values(channels, alpha)

In [4]:
def rotate_image(image, angle):
    if angle == 0: return image
    height, width = image.shape[:2]
    rot_mat = cv2.getRotationMatrix2D((width/2, height/2), angle, 0.9)
    result = cv2.warpAffine(image, rot_mat, (width, height), flags=cv2.INTER_LINEAR)
    return result

def rotate_point(pos, img, angle):
    if angle == 0: return pos
    x = pos[0] - img.shape[1]*0.4
    y = pos[1] - img.shape[0]*0.4
    newx = x*cos(radians(angle)) + y*sin(radians(angle)) + img.shape[1]*0.4
    newy = -x*sin(radians(angle)) + y*cos(radians(angle)) + img.shape[0]*0.4
    
    return int(newx), int(newy), pos[2], pos[3]

Settings for the cascade classifier

In [5]:

settings = {
    'scaleFactor': 1.3, 
    'minNeighbors': 3, 
    'minSize': (50, 50)
}

In [6]:
def main():
    vc = cv2.VideoCapture(0)

    if vc.isOpened(): 
        rval, frame = vc.read()
    else:
        rval = False
    while rval:
        gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        
        
        # This allows for the face to be turned slightly and still be detected
        for angle in [0, -25, 25]:
            rimg = rotate_image(gray, angle)
            detected = faceCascade.detectMultiScale(rimg, **settings)
            if len(detected):
                # faces is a rotation of the detected points 
                faces = [rotate_point(detected[-1], gray, -angle)]
                break

        
        SRC = frame
        # If faces are detected
        if (len(detected)):
            # for every x, y, width, and height in faces
            for (x, y, w, h) in faces:
                    # draw a rectangle on the face
                    cv2.rectangle(SRC, (x, y), (x+w, y+h), (0, 255, 0), 2)
                    # Put the mustache in the correct area on the face
                    SRC = put_mustache(mst,SRC,x,y+7,w,h) #Tends to go a bit high, let's pull it down a bit

        
        cv2.imshow("Stache!", SRC)
        
        
        # Exit functionality - press any key to exit laptop video
        key = cv2.waitKey(20)
        if key > 0: # Exit by pressing any key
            # Destroy windows 
            cv2.destroyAllWindows()
            
            # Make sure window closes on OSx
            for i in range (1,5):
                cv2.waitKey(1)
            return SRC
        
        # Read next frame
        time.sleep(0.05)             # control framerate for computation - default 20 frames per sec
        rval, frame = vc.read()

In [8]:
if __name__ == "__main__":
    main()